# ABS Monthly Arrivals Departures 3401

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# local imports
import abs_data_capture as adc
import plotting as p

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [3]:
def get_data() -> tuple[dict[str, pd.DataFrame], pd.DataFrame, str, pd.Timestamp]:
    """Capture relevant ABS data, set-up plotting output directories"""

    landing_page_ = adc.AbsLandingPage(
        theme="industry",
        parent_topic="tourism-and-transport",
        topic="overseas-arrivals-and-departures-australia",
    )
    abs_dict_ = adc.get_abs_data(landing_page_, verbose=False)
    source_, chart_dir, _cat_id, meta_ = adc.get_fs_constants(abs_dict_, landing_page_)
    recent_, _plot_times, _plot_tags = adc.get_plot_constants(meta_)

    # plotting set-up
    p.clear_chart_dir(chart_dir)
    p.set_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")

    return abs_dict_, meta_, source_, recent_

In [4]:
abs_dict, meta, source, recent = get_data()

Caution: Could not find the 'Index' sheet in 3401013.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401014.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401015.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401016.xlsx. File not included
Caution: Could not find the 'Index' sheet in 3401017.xlsx. File not included


In [5]:
# data up-to-date to (year-month) ...
print(abs_dict["1"].index[-1])

2024-02


## Total Overseas Arrivals and Departures

In [6]:
def headline() -> None:
    """Headline charts"""

    plotable = (
        # [did, table]
        ("Number of movements ;  Total Departures ;", "2"),
        ("Number of movements ;  Total Arrivals ;", "1"),
    )

    plt_df = pd.DataFrame()
    for did, table in plotable:
        search = {
            did: adc.metacol.did,
            table: adc.metacol.table,
            "Original": adc.metacol.stype,
        }
        series_id, units = adc.find_id(meta, search)
        plt_df[did.split(";")[-2]] = abs_dict[table][series_id]

    plt_df, units = p.recalibrate(plt_df, units)

    p.line_plot(
        plt_df,
        title="Total Overseas Arrivals and Departures",
        starts=(None, pd.Period(recent, freq="M")),
        ylabel=units,
        rfooter=source,
        lfooter="Australia. ",
        show=SHOW,
    )

In [7]:
headline()

## Individual overseas movements

In [8]:
def plot_movements() -> None:
    """Headline charts"""

    tables = "1", "2"
    arrivals = meta.loc[meta[adc.metacol.table] == tables[0], adc.metacol.did]
    departures = meta.loc[meta[adc.metacol.table] == tables[1], adc.metacol.did]
    movements = arrivals, departures

    for movement, table in zip(movements, tables):
        data = abs_dict[table]
        for did in movement:
            search = {
                did: adc.metacol.did,
                table: adc.metacol.table,
                "Original": adc.metacol.stype,
            }
            series_id, units = adc.find_id(meta, search, exact=True, verbose=False)
            series, units = p.recalibrate(data[series_id], units)

            p.line_plot(
                series,
                title=did.split(";")[-2],
                starts=(None, pd.Period(recent, freq="M")),
                ylabel=units,
                rfooter=source,
                lfooter="Australia. ",
                show=SHOW,
            )

In [9]:
plot_movements()

## Migration proxy

In [10]:
def migration_proxy() -> None:
    """Migration proxy charts"""

    plotable = (
        # [did, table]
        ("Number of movements ;  Permanent and Long-term Arrivals ;", "1"),
        ("Number of movements ;  Permanent and Long-term Departures ;", "2"),
    )

    plt_df = pd.DataFrame()
    for did, table in plotable:
        search = {
            did: adc.metacol.did,
            table: adc.metacol.table,
            "Original": adc.metacol.stype,
        }
        series_id, units = adc.find_id(meta, search)
        plt_df[did.split(";")[-2]] = abs_dict[table][series_id]

    YEAR = 12  # months
    plt_df = plt_df.rolling(window=YEAR, min_periods=YEAR).sum()
    net = plt_df[plt_df.columns[0]] - plt_df[plt_df.columns[1]]
    net, units = p.recalibrate(net, units)

    p.line_plot(
        net,
        title="Net permanent and long-term overseas movements\n"
        f"(arrivals - departures), {YEAR}-month rolling sum",
        starts=(None, pd.Period(recent, freq="M")),
        ylabel=units,
        rfooter=source,
        lfooter="Australia. ",
        y0=True,
        show=SHOW,
    )

In [11]:
migration_proxy()

## Finished

In [12]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Apr 16 2024 11:42:08

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

pandas    : 2.2.2
matplotlib: 3.8.4

Watermark: 2.4.3



In [13]:
print("Finished")

Finished
